In [1]:
def load_input(filename):
    hotspring_lines = []

    with open(filename) as f:
        for line in f:
            row, cgods = line.strip().split(" ")
            cgods_list = cgods.split(",")
            cgods_list = [int(cgod) for cgod in cgods_list]
            hotspring_lines.append({"row": row, "cgods": cgods_list})
    return hotspring_lines


hot_springs_lines = load_input("input.txt")
hot_springs_lines[:5]

[{'row': '##???#??#?????????#?', 'cgods': [11, 6]},
 {'row': '???.#??#.???', 'cgods': [1, 1, 2, 1]},
 {'row': '#????#?.???????.', 'cgods': [3, 3, 4]},
 {'row': '.#?????####.?.#?', 'cgods': [1, 1, 5, 1]},
 {'row': '?#?????????#?#?', 'cgods': [3, 2, 1, 1, 1]}]

In [2]:
def is_valid(hotspring_line):
    # .#.#.### 1,1,3
    # print(hotspring_line)
    row, cgods = hotspring_line["row"], hotspring_line["cgods"].copy()
    in_sequence = False
    for i in range(len(row)):
        spring = row[i]
        if spring == "#":
            if len(cgods) == 0:
                return False
            in_sequence = True
            cgods[0] -= 1
            if cgods[0] < 0:
                return False
        elif spring == ".":
            if in_sequence:
                if cgods[0] == 0:
                    cgods.pop(0)
                    in_sequence = False
                    continue
                else:
                    return False

    if in_sequence:
        if cgods[0] == 0:
            cgods.pop(0)
        else:
            return False

    return not bool(cgods)


def test_is_valid():
    assert is_valid({"row": ".....", "cgods": []}) == True
    assert is_valid({"row": ".....", "cgods": [1]}) == False
    assert is_valid({"row": "....#", "cgods": [1]}) == True
    assert is_valid({"row": ".....", "cgods": [1, 1]}) == False
    assert is_valid({"row": "..#..", "cgods": [1]}) == True
    assert is_valid({"row": "..#..", "cgods": [1, 1]}) == False
    assert is_valid({"row": "..##.", "cgods": [1]}) == False
    assert is_valid({"row": "#.##.", "cgods": [1, 2]}) == True
    assert is_valid({"row": "####.", "cgods": [1, 3]}) == False
    assert is_valid({"row": "#####", "cgods": [5]}) == True


test_is_valid()

In [3]:
def possible_configurations(hotspring_line):
    row, cgods = hotspring_line["row"], hotspring_line["cgods"]

    all_possible_configurations = []
    question_marks = row.count("?")
    for i in range(2**question_marks):
        binary = bin(i)[2:].zfill(question_marks)
        configuration = row
        for j in range(question_marks):
            configuration = configuration.replace("?", binary[j], 1)
        configuration = configuration.replace("0", ".")
        configuration = configuration.replace("1", "#")
        all_possible_configurations.append(configuration)

    # print(all_possible_configurations)

    sum = 0
    for configuration in all_possible_configurations:
        if is_valid({"row": configuration, "cgods": cgods}):
            sum += 1

    return sum


def test_possible_configurations():
    test = load_input("test.txt")
    # print(test)
    assert possible_configurations(test[0]) == 1
    assert possible_configurations(test[1]) == 4
    assert possible_configurations(test[2]) == 1
    assert possible_configurations(test[3]) == 1
    assert possible_configurations(test[4]) == 4
    assert possible_configurations(test[5]) == 10


test_possible_configurations()

sum([possible_configurations(row) for row in hot_springs_lines])

6949